In [2]:
from lime import lime_image
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries

In [3]:
model = tf.keras.models.load_model('../assets/best_model.h5')

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-06-05 11:58:18.142399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-05 11:58:18.142538: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
target = [
    'beagle', 'cocker_spaniel', 'golden_retriever',
    'maltese', 'pekinese', 'pomeranian', 'poodle',
    'samoyed', 'shih_tzu', 'white_terrier']


In [5]:
def img_preprocess(img_path):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img = img.convert('RGB')
    img = np.array(img)
    img = img / 255
    return img

In [15]:
def explain_image(img_path):
    img = img_preprocess(img_path)
    explainer = lime_image.LimeImageExplainer()

    # 이미지를 슈퍼픽셀로 분할하는 알고리즘 설정
    # quickshift, slic, felzenswalb 등이 존재
    segmenter = SegmentationAlgorithm('slic',
                                    n_segments=100,  # 이미지 분할 조각 개수
                                    compactnes=1,  # 유사한 파트를 합치는 함수
                                    sigma=1)  # 스무딩 역할: 0과 1사이의 float

    explanation = explainer.explain_instance(img,
                                            classifier_fn=model.predict,  # 10개 class 확률 반환
                                            top_labels=1,  # 확률 기준 1위
                                            num_samples=1000,  # sample space
                                            segmentation_fn=segmenter)  # 분할 알고리즘                                            
    
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=True)
    
    plt.subplot(2, 2, 1)
    plt.title(f'Significant heatmap')
    plt.rcParams['figure.figsize'] = (224, 224)
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.imshow(img, alpha=0.5)


    np_img = np.array([img])
    pred = model.predict(np_img)
    # 예측한 상위 두 클래스만 출력
    pred_list = [(pred[0][i], target[i]) for i in range(len(target))]
    pred_list = sorted(pred_list)
    for t in pred_list[-2:]:
        print(f'{t[1]} : {t[0] * 100 : .2f}%')
    pred_idx = np.argmax(pred)
    print(f'This image is {pred[0][pred_idx] * 100 :.2f}% {target[pred_idx]}')

    ind = explanation.top_labels[0]
    dict_heatmap = dict(explanation.local_exp[ind])
    
    heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
    
    plt.title("Weight of each segment")
    plt.rcParams['figure.figsize'] = (224, 224)
    plt.subplot(2, 2, 2)
    plt.imshow(heatmap, cmap='RdBu', vmin=-heatmap.max(), vmax=heatmap.max())
    plt.colorbar()


In [14]:
img_path = '../test_img/jjang.jpeg'
explain_image(img_path)
# The answer to this image is shih-tzu


100%|██████████| 1000/1000 [00:21<00:00, 46.29it/s]


maltese :  0.02%
shih_tzu :  99.98%
This image is 99.98% shih_tzu


In [13]:
img_path = '../test_img/asd.jpeg'
explain_image(img_path)

# The answer to this image is white terrier

100%|██████████| 1000/1000 [00:21<00:00, 47.06it/s]


pekinese :  0.00%
white_terrier :  100.00%
This image is 100.00% white_terrier


In [12]:
img_path = '../test_img/2.jpeg'
explain_image(img_path)

# The answer to this image is maltepoo (Maltepoo is a mix of poodle and maltese)


100%|██████████| 1000/1000 [00:20<00:00, 48.02it/s]


poodle :  28.78%
maltese :  66.34%
This image is 66.34% maltese


In [10]:
img_path = '../test_img/mint3.jpeg'
explain_image(img_path)

# The answer to this image is white terrirer, but not pure

100%|██████████| 1000/1000 [00:20<00:00, 47.86it/s]


maltese :  23.08%
white_terrier :  76.92%
This image is 76.92% white_terrier
